<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Deprecated Display Routines</strong>
</div></div>

# 1. Display a Layout of the Gaussian Elimination Algorithm

The code int this notebook is intended as a debugging aid when writing a Gaussian Elimination Solver:<br>
starting with a matrix $A$,
* we find an elimination matrix $E_1$ and update the matrix to $A1 = E_1 A$. Save the computation in the augmented matrix [E_1 A_1]
* for each successive step, $E_i, A_i$, update the augmented matrix with [previous_augmented_matrix ; E_i A_i]
Save the pivot locations in the same way.

When the algorithm completes, we have
* the original array $A$
* the computations in another array
$$Computations =\left( \begin{array}{rr}
  E_1 & A_1 \\
  E_2 & A_2 \\
  \dots & \dots
\end{array} \right)
$$
and the pivot index pairs $p_i$ in an array or a list $( p_1, p_2, \dots p_i \dots )$.

The `ge_layout` function can now be used to display the computation.

## 1.1 Basic Display Function: build a table from the matrices (DEPRECATED)

**Remark**>br>
With the advent of the CTAN nicematrix package, the following functions are deprecated.<br>
The replacement package `itikz` generates a matrix of matrices with proper alignment and decorations.

____
Let's start with a simple function that allows us to create titles: it outputs an
HTML `<div>` containing text of a given size and color,<br>
as well as justification of the text. See https://www.w3schools.com/Tags/tag_div.asp

In [ ]:
function title( txt; sz=25,color="blue",justify="left")
    t = """<br><div style="width:30cm;color:$(color);text-align:$(justify);font-size:$(sz)px;">$(txt)</div>"""
    display(HTML(t))
end
#title("A title", justify="center", sz=40)

**Remark:** This function is now part of GenLinAlgProblems as function `pr()`

Next, we define a first version of the `ge_layout` function: it will accept the matrices $A$ and $Computations$,
and write these into one big table.

The routine alternates colors in the table to allow identification of the matrices,
and uses a third input (the function `to_string` to convert a table entry to an HTML string.

In [ ]:
using PrettyTables, Printf
#using LaTeXStrings
# --------------------------------------------------------------------------------------------------
"""
pt_frac(value)

HTML format of Julia Fractions

# Example:
```julia-repl
julia> pt_frac(-3//5)
3 / 5
```
"""
function pt_frac(value)
    n,d=numerator(value),denominator(value)
    d == 1 ? "$n" : "$n &frasl; $d"
end
# --------------------------------------------------------------------------------------------------
"""
ge_layout( A, layers )

Display a computational layout for a Gaussian Elimination Computation

# Arguments
- `n::Array`:         an array of integers or rational numbers
- `layers::Array`:    an array of the computations
- `to_str::Function`: a function converting a matrix entry to a string

# Example:
```julia-repl
julia> ge_layout( A, [ E1 A1; E2 A2; E3 A3], to_str = pt_real") # E_{i} is the i'th elemination matrix, A_i = E_i * A_{i-1}
the layout
```
"""
function ge_layout( A, layers, to_str = pt_frac )
    n  = size(A, 1)
    tf = HtmlTableFormat(css=""".verticalline { border-right: 1px solid black; height: 100%; }""" )

    # define a vertical divider (here we use '|'), and rearrange the table entries to put it in place
    #sp = fill("&nbsp;", n+size(layers,1),1 )
    sp = fill("<span class=verticalline></span>", n+size(layers,1), 1)
    MA = vcat( hcat( fill("", n,n), map(to_str, A)),
                     map( to_str,  layers))
    M   = [MA[:,1:n] sp MA[:,n+1:end]]

    # Now add the background colors for the matrices
    bg_gray_1    = HtmlHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j>n,  HtmlDecoration(background = "#F0F0F0", color = "blue"  ))
    bg_gray_2    = HtmlHighlighter((data,i,j)->isodd( (i-1)÷n+1)        && j>n,  HtmlDecoration(background = "#FBFAFA", color = "blue"  ))

    bg_yellow_1  = HtmlHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j<=n, HtmlDecoration(background = "#FDFAD4", color = "blue"  ))
    bg_yellow_2  = HtmlHighlighter((data,i,j)->i>n && isodd( (i-1)÷n+1) && j<=n, HtmlDecoration(background = "#F6EFA1", color = "blue"  ))
    bg_white     = HtmlHighlighter((data,i,j)->i<=n                     && j<=n, HtmlDecoration(background = "#FFFFFF", color = "blue"  ))

    # create and return the table
    pretty_table( M, noheader = true, highlighters=(bg_white,bg_gray_1,bg_gray_2,bg_yellow_1,bg_yellow_2), tf=tf, backend = :html )
end;

## 1.2 Gaussian Elimination for a given matrix $A$

Here, we do the computation 'by hand'

In [ ]:
# Look at the following matrix
A  = [ 1 3 1 4;
      -2 5 5 6;
       1 3 1 5;
       2 2 8 2
     ]
# The GE algorithm results in the following steps
p1 = (1,1)
E1 = [ 1 0 0 0;
       2 1 0 0;
      -1 0 1 0;
      -2 0 0 1
    ]
A1 = E1 * A

p2 = (2,2)
E2 = [ 1 0 0 0;
       0 1 0 0;
       0 0 1 0;
       0 4//11 0 1
    ]
A2 = E2 * A1

p3 = (4,3)
E3 = [ 1 0 0 0;
       0 1 0 0;
       0 0 0 1;
       0 0 1 0
    ]
A3 = E3 * A2

# Let's display the computation
title("Gaussian Elimination Example")
ge_layout( A, [E1 A1; E2 A2; E3 A3])

Note that we used exact arithmetic, and displayed the resulting fractions.<br>
We could convert the fractions to floating point numbers by passing a function
into the `ge_layout` routine: `value->@sprintf( "%8.2f", value)`
converts an entry value to a string of width 8 characters, with 2 decimal places.

Note the width should be at least number of decimal places + 6

In [ ]:
title("Gaussian Elimination Example<br>with rounding")
ge_layout(A, [E1 A1; E2 A2; E3 A3], value->@sprintf( "%8.2f", value) )

## 1.3 Modify ge_layout to highlight the pivots

Let's create a somewhat more sophisticated version of `ge_layout`: add a font color for the pivots

In [ ]:
function ge_layout( A, layers, pivots; to_str = pt_frac, extra_cols=0 )
    n,m  = size(A)
    m   -= extra_cols
    # table entry indices for the pivots; note j+m+1 to account for the inserted separator |
    # this tuple of index pairs will have corresponding entries colored magenta
    p    = ( ((l-1)*n+i, (j+n+1))  for (l,(i,j)) in enumerate(pivots))

    hl = (
        # Pivot positions; alternate matrix backgrounds with iseven, isodd
        HtmlHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j>n && (i,j) in p,
            HtmlDecoration(background = "#F0F0F0", color = "magenta" )),
        HtmlHighlighter((data,i,j)->isodd( (i-1)÷n+1)        && j>n && (i,j) in p,
            HtmlDecoration(background = "#FBFAFA", color = "magenta" )),

        HtmlHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j<=n && (i,j) in p,
            HtmlDecoration(background = "#FDFAD4", color = "magenta" )),
        HtmlHighlighter((data,i,j)->i>n && isodd( (i-1)÷n+1) && j<=n && (i,j) in p,
            HtmlDecoration(background = "#F6EFA1", color = "magenta" )),
        HtmlHighlighter((data,i,j)->i<=n                     && j<=n && (i,j) in p,
            HtmlDecoration(background = "#FFFFFF", color = "magenta" )),

        # non pivot positions
        HtmlHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j>n,
            HtmlDecoration(background = "#F0F0F0", color = "blue" )),
        HtmlHighlighter((data,i,j)->isodd( (i-1)÷n+1)        && j>n,
            HtmlDecoration(background = "#FBFAFA", color = "blue" )),

        HtmlHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j<=n,
            HtmlDecoration(background = "#FDFAD4", color = "blue" )),
        HtmlHighlighter((data,i,j)->i>n && isodd( (i-1)÷n+1) && j<=n,
            HtmlDecoration(background = "#F6EFA1", color = "blue" )),
        HtmlHighlighter((data,i,j)->i<=n                     && j<=n,
            HtmlDecoration(background = "#FFFFFF", color = "blue" ))
    )

    tf  = HtmlTableFormat(css=""".verticalline { border-right: 1px solid black; height: 100%; }""" ) # dividing line css
    if length( layers ) > 0
        MA  = vcat( hcat( fill("", n,n), map(to_str, A)), # *  A
                    map( to_str, layers))                 # Ei Ai

        sp  = fill("<span class=verticalline></span>", n+size(layers,1), 1)  # dividing line
        if extra_cols > 0 # need a divider in cols n+1 and m+n+2
            M   = [MA[:,1:n] sp MA[:,n+1:n+m] sp MA[:, n+1+m:end]]
        else
            M   = [MA[:,1:n] sp MA[:,n+1:end]]
        end
    else
        M = hcat( fill("", n,n), fill("<span class=verticalline></span>", n,1), map(to_str, A))
    end

    pretty_table( M, noheader = true, highlighters=hl, tf=tf, backend = :html )
end
ge_layout( A, [E1 A1; E2 A2; E3 A3], (p1,p2,p3), to_str=value->@sprintf( "% 8.2f", value) )

## 1.4 Add a slider to animate the computation

As an experiment, let's add a slider to show the computations one set of $E_i A_i$ matrices at a time.

Note: not really worth it; the display flickers (Also WebIO has a bug which requires reloading the page to actually work)

In [ ]:
# Remark: for me, this works in Chromium but not in Firefox
using Interact

ge_steps  = [E1 A1; E2 A2; E3 A3]
N_steps   = Int64(size(ge_steps,1)/size(A,1))
step_ctrl = slider(0:N_steps,value=0,label="Step")
display.([step_ctrl] )

function show_step(i)
#h = on(step_ctrl) do i
    IJulia.clear_output(true)
    display.([step_ctrl])
    title( "Gaussian Elimination Step $i")
    ge_layout( A, ge_steps[1:i*size(A,1),:],(p1,p2,p3), to_str=value->@sprintf( "% 8.2f", value) )
end

#Interact.@on show_step( &step_ctrl )
on( show_step, step_ctrl)

step_ctrl[] = 0;

Alternatively, we could use the sleep function to put in a delay...

In [ ]:
# Alternatively, we could use a timer....
ge_steps  = [E1 A1; E2 A2; E3 A3]
N_steps   = Int64(size(ge_steps,1)/size(A,1))

title( "GE step 0")
ge_layout(A, [], (p1,p2,p3) )
for i = 1:N_steps
    sleep(4)
    IJulia.clear_output(true)
    title( "GE step $i")
    ge_layout(A, ge_steps[1:i*size(A,1),:], (p1,p2,p3), to_str=value->@sprintf( "% 8.2f", value) )
end

# 2. Coding Assignment

Create a function that takes a matrix as an input, and returns a corresponding row echelon form.<br>
You might consider using `ge_layout` for debugging your code.

In [ ]:
function row_echelon_form(A)
    M,N = size(A)
    println("The matrix A has size $M x $N")
    display(A)

    # make a copy of A: we will modify the copy, not A itself
    R = copy(A)
    # < your code here > ***************************************************** <<<<<< 
    R
end

In [ ]:
row_echelon_form(A);

## 2.1 Some code building blocks

### 2.1.1 Submatrices

In [ ]:
println("A vector:  row_index=1")
A[1, 1:3]

In [ ]:
println( "A matrix (row vector): row_index=1:1")
A[1:1, 1:3]

In [ ]:
println( "A column vector: transpose of a row vector")
A[1:1,1:3]'

### 2.1.2 Conditional Statements

In [ ]:
if 3 > 1 && 8 < 9
    println( "if branch")
else
    println( "else branch")
end

### 2.1.3 Loop statements `for loops`

In [ ]:
for i in 1:10
    print( " $i")
end

In [ ]:
# premature temination of the loop
for i in 1:10
    print( " $i")
    if i > 4
        break
    end
end

In [ ]:
# skip some code inside a loop
for i in 1:10
    if i > 3 && i < 8
        continue
    end
    print( " $i")
end

# 2.2 Needed functionality: code snippets you will need to write

> * given  a current row and column, find a pivot
> * interchange rows in a matrix
> * use a pivot row to produce a zero at some location in a row (the elimination operation)
> * possibly scale a row

In [ ]:
display(A)
# R_2 <- R_2 + 2 R_1
A[2:2, : ]  = A[2:2, :] + 2 * A[1:1, :]
A

In [ ]:
# R_3 <-> R_4
A[3:3,:], A[4:4,:] = A[4:4,:], A[3:3,:]
A

In [ ]:
# If we want to construct an elementary operation matrix:
if false  # temporarily set to true if you have not yet downloaded the LinearAlgebra library
    using Pkg
    Pkg.add("LinearAlgebra")
end

using LinearAlgebra

In [ ]:
display( I )     # an identity matrix of arbitrary size
display( 1.0I )  # a floating point identity matrix of arbitrary size
display("-----------------------------------")
E = Matrix(1.0I, 4, 4) # A 4x4 floating point identity matrix
display(E)

In [ ]:
E[2:end,1]=[1 3 4]
E

In [ ]:
title("Have fun!")

# 3. Having fun: what if we have errors in A?

In [ ]:
using Measurements, LinearAlgebra

## 3.1 Errors in the matrix

In [ ]:
A = randn((3,3)) .± 0.001randn((3,3))
display( A )
b=[1.; 1.; 0.]
x = A \ b
display(x)
norm(A*x-b)

In [ ]:
# However, if the system is inconsistent, we have
A = [randn((2,3)); 0 0 0]
A = A .± 0.001randn((3,3))

In [ ]:
A \ b

## 3.2 Errors in the right hand side

In [ ]:
A = randn((3,3))
display( A )
b=[1.; 1.; 0.] .± 0.01randn((3,1))
x = A \ b
display(x)
norm(A*x-b)

## 3.3 Errors in both the matrix and the right hand side

In [ ]:
A = randn((3,3)) .± 0.1randn((3,3))
display( A )
b=[1.; 1.; 0.] .± 0.1randn((3,1))
x = A \ b
display(x)
norm(A*x-b)

## 3.4 What if the system is almost inconsistent

In [ ]:
A = [randn((2,3)); 0 0 1e-10]
b = [1;1;1]
x = A \ b
display(A); display(b); display(x)
norm(A*x-b)